In [29]:
#imports

import os
import sys
import pandas as pd
import numpy as np
import time
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta
import datetime
import re
import joblib
from urllib.error import HTTPError

import eli5
from eli5.sklearn import PermutationImportance
from matplotlib.colors import ListedColormap

import warnings
from sklearn import tree
import statsmodels.formula.api as smf
from statsmodels.miscmodels.ordinal_model import OrderedModel
import sklearn
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor, AdaBoostRegressor
from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, validation_curve, learning_curve
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor, ElasticNet, BayesianRidge, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import f1_score, make_scorer, classification_report, precision_score, accuracy_score, recall_score
from sklearn.preprocessing import MinMaxScaler, scale, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb
from sklearn.feature_selection import RFE

from scipy.stats import poisson
from matplotlib import pyplot as plt

from bs4 import BeautifulSoup

import requests
import itertools as it

## Preprocessing

In [30]:
#form

def homeFormP(team, games, endDate, df):
    home = df.query("home == @team and date < @endDate").tail(games)
    home['results'] = home['results'].replace(['H', 'D', 'A'], [3, 1, 0])
    return home['results'].sum()

def awayFormP(team, games, endDate, df):
    away = df.query("away == @team and date < @endDate").tail(games)
    away['results'] = away['results'].replace(['H', 'D', 'A'], [0, 1, 3])
    return away['results'].sum()

def homeForm(team, games, df):
    home = df.query("home == @team").tail(games)
    home['results'] = home['results'].replace(['H', 'D', 'A'], [3, 1, 0])
    return home

def awayForm(team, games, df):
    away = df.query("away == @team").tail(games)
    away['results'] = away['results'].replace(['H', 'D', 'A'], [0, 1, 3])
    return away

def addForm(df):
    home_5_form = []
    away_5_form = []
    home_3_form = []
    away_3_form = []
    home_1_form = []
    away_1_form = []
    
    for i in df.index:
        home_5_form.append(homeFormP(df.iloc[i]['home'], 5, df.iloc[i]['date'], df))
        away_5_form.append(awayFormP(df.iloc[i]['away'], 5, df.iloc[i]['date'], df))
        home_3_form.append(homeFormP(df.iloc[i]['home'], 3, df.iloc[i]['date'], df))
        away_3_form.append(awayFormP(df.iloc[i]['away'], 3, df.iloc[i]['date'], df))
        home_1_form.append(homeFormP(df.iloc[i]['home'], 1, df.iloc[i]['date'], df))
        away_1_form.append(awayFormP(df.iloc[i]['away'], 1, df.iloc[i]['date'], df))
    
    df['home_5_form'] = pd.DataFrame(home_5_form)
    df['away_5_form'] = pd.DataFrame(away_5_form)
    df['home_3_form'] = pd.DataFrame(home_3_form)
    df['away_3_form'] = pd.DataFrame(away_3_form)
    df['home_1_form'] = pd.DataFrame(home_1_form)
    df['away_1_form'] = pd.DataFrame(away_1_form)
    return df

In [31]:
#extra features
def last_n_var(var, against, team, place, date, n, df):
    if against == 'for':
        if place == 'home':
            games = homeForm(team, 100, df).query('date < @date')['home_' + var]
            
        if place == 'away':
            games = awayForm(team, 100, df).query('date < @date')['away_' + var]
            
        if place == 'any':
            games = form(team, 100, df).query('date < @date').tail(n)
            home = games.query('home == @team')['home_' + var].sum()
            away = games.query('away == @team')['away_' + var].sum()
            return (home + away) / (len(games.tail(n)))
        
    if against == 'against':
        if place == 'home':
            games = homeForm(team, 100, df).query('date < @date')['away_' + var]
            
        if place == 'away':
            games = awayForm(team, 100, df).query('date < @date')['home_' + var]
            
        if place == 'any':
            games = form(team, 100, df).query('date < @date').tail(n)
            home = games.query('home == @team')['away_' + var].sum()
            away = games.query('away == @team')['home_' + var].sum()
            return (home + away) / (len(games.tail(n)))
    return games.tail(n).mean()

def runningAverage(df):
    data = df.copy()
    names = ['last_3_goals_for_home', 'last_3_goals_against_home', 'last_3_goals_for_away', 'last_3_goals_against_away',
             'last_5_goals_for_home', 'last_5_goals_against_home', 'last_5_goals_for_away', 'last_5_goals_against_away',
             #'last_3_xg_for_home', 'last_3_xg_against_home', 'last_3_xg_for_away', 'last_3_xg_against_away',
             #'last_5_xg_for_home', 'last_5_xg_against_home', 'last_5_xg_for_away', 'last_5_xg_against_away',
             #'last_3_poss_for_home', 'last_3_poss_against_home', 'last_3_poss_for_away', 'last_3_poss_against_away',
             #'last_5_poss_for_home', 'last_5_poss_against_home', 'last_5_poss_for_away', 'last_5_poss_against_away',
             'last_3_shot_for_home', 'last_3_shot_against_home', 'last_3_shot_for_away', 'last_3_shot_against_away',
             'last_5_shot_for_home', 'last_5_shot_against_home', 'last_5_shot_for_away', 'last_5_shot_against_away',
             'last_3_sot_for_home', 'last_3_sot_against_home', 'last_3_sot_for_away', 'last_3_sot_against_away',
             'last_5_sot_for_home', 'last_5_sot_against_home', 'last_5_sot_for_away', 'last_5_sot_against_away',
             #'last_3_pass_for_home', 'last_3_pass_against_home', 'last_3_pass_for_away', 'last_3_pass_against_away',
            ]#'last_5_pass_for_home', 'last_5_pass_against_home', 'last_5_pass_for_away', 'last_5_pass_against_away']

    for i in names:
        name = i
        i = i.split('_')
        var = i[2]
        against = i[3]
        place = i[4]
        n = int(i[1])
        if place == 'home':
            col = data.apply(lambda row: last_n_var(var, against, row.home, 'home', row.date, n, df), axis = 1) 
        if place == 'away':
            col = data.apply(lambda row: last_n_var(var, against, row.away, 'away', row.date, n, df), axis = 1)
        col = pd.DataFrame(col, columns = [name])
        data = data.join(col)
    return data

def ratio(df):
    df['last_3_goals_home_att_away_def'] = df['last_3_goals_for_home'] * df['last_3_goals_against_away']
    df['last_3_goals_home_def_away_att'] = df['last_3_goals_against_home'] * df['last_3_goals_for_away']
    df['last_5_goals_home_att_away_def'] = df['last_5_goals_for_home'] * df['last_5_goals_against_away']
    df['last_5_goals_home_def_away_att'] = df['last_5_goals_against_home'] * df['last_5_goals_for_away']
    #df['last_3_xg_home_att_away_def'] = df['last_3_xg_for_home'] * df['last_3_xg_against_away']
    #df['last_3_xg_home_def_away_att'] = df['last_3_xg_against_home'] * df['last_3_xg_for_away']
    #df['last_5_xg_home_att_away_def'] = df['last_5_xg_for_home'] * df['last_5_xg_against_away']
    #df['last_5_xg_home_def_away_att'] = df['last_5_xg_against_home'] * df['last_5_xg_for_away']
    #df['last_3_poss_ratio'] = df['last_3_poss_for_home'] / df['last_3_poss_for_away']
    #df['last_5_poss_ratio'] = df['last_5_poss_for_home'] / df['last_5_poss_for_away']
    df['last_3_shot_home_att_away_def'] = df['last_3_shot_for_home'] * df['last_3_shot_against_away']
    df['last_3_shot_home_def_away_att'] = df['last_3_shot_against_home'] * df['last_3_shot_for_away']
    df['last_5_shot_home_att_away_def'] = df['last_5_shot_for_home'] * df['last_5_shot_against_away']
    df['last_5_shot_home_def_away_att'] = df['last_5_shot_against_home'] * df['last_5_shot_for_away']
    df['last_3_sot_home_att_away_def'] = df['last_3_sot_for_home'] * df['last_3_sot_against_away']
    df['last_3_sot_home_def_away_att'] = df['last_3_sot_against_home'] * df['last_3_sot_for_away']
    df['last_5_sot_home_att_away_def'] = df['last_5_sot_for_home'] * df['last_5_sot_against_away']
    df['last_5_sot_home_def_away_att'] = df['last_5_sot_against_home'] * df['last_5_sot_for_away']
    #df['last_3_pass_home_att_away_def'] = df['last_3_pass_for_home'] * df['last_3_pass_against_away']
    #df['last_3_pass_home_def_away_att'] = df['last_3_pass_against_home'] * df['last_3_pass_for_away']
    #df['last_5_pass_home_att_away_def'] = df['last_5_pass_for_home'] * df['last_5_pass_against_away']
    #df['last_5_pass_home_def_away_att'] = df['last_5_pass_against_home'] * df['last_5_pass_for_away']
    df['home_goals_attack'] = df['home_avg_for_goals'] * df['away_avg_against_goals']
    df['away_goals_attack'] = df['away_avg_for_goals'] * df['home_avg_against_goals']
    df['home_shot_attack'] = df['home_avg_for_shot'] * df['away_avg_against_shot']
    df['away_shot_attack'] = df['away_avg_for_shot'] * df['home_avg_against_shot']
    df['home_sot_attack'] = df['home_avg_for_sot'] * df['away_avg_against_sot']
    df['away_sot_attack'] = df['away_avg_for_sot'] * df['home_avg_against_sot']
    df['home_corner_attack'] = df['home_avg_for_corner'] * df['away_avg_against_corner']
    df['away_corner_attack'] = df['away_avg_for_corner'] * df['home_avg_against_corner']
        
    return df

def leagueAverage(var, against, team, place, date, df):
    if against == 'for':
        if place == 'home':
            games = homeForm(team, 100, df).query('date < @date')['home_' + var]
            
        if place == 'away':
            games = awayForm(team, 100, df).query('date < @date')['away_' + var]
        
    if against == 'against':
        if place == 'home':
            games = homeForm(team, 100, df).query('date < @date')['away_' + var]
            
        if place == 'away':
            games = awayForm(team, 100, df).query('date < @date')['home_' + var]

    return games.mean()

def average(df):
    data = df.copy()
    names = ['home_avg_for_goals', 'home_avg_against_goals', 'away_avg_for_goals', 'away_avg_against_goals',
            'home_avg_for_yellow', 'home_avg_against_yellow', 'away_avg_for_yellow', 'away_avg_against_yellow',
            'home_avg_for_red', 'home_avg_against_red', 'away_avg_for_red', 'away_avg_against_red',
            'home_avg_for_foul', 'home_avg_against_foul', 'away_avg_for_foul', 'away_avg_against_foul',
            'home_avg_for_shot', 'home_avg_against_shot', 'away_avg_for_shot', 'away_avg_against_shot',
            'home_avg_for_sot', 'home_avg_against_sot', 'away_avg_for_sot', 'away_avg_against_sot',
            'home_avg_for_corner', 'home_avg_against_corner', 'away_avg_for_corner', 'away_avg_against_corner']
    
    for i in names:
        name = i
        i = i.split('_')
        var = i[3]
        against = i[2]
        place = i[0]
        if place == 'home':
            col = data.apply(lambda row: leagueAverage(var, against, row.home, 'home', row.date, df), axis = 1) 
        if place == 'away':
            col = data.apply(lambda row: leagueAverage(var, against, row.away, 'away', row.date, df), axis = 1)
        col = pd.DataFrame(col, columns = [name])
        data = data.join(col)
    return data

In [32]:
#league position
def pos(df, team, table):
    index = table.index
    condition = table['teams'] == team
    position = index[condition][0] + 1
    return position

def avgPos(df):
    leagueTable = table(df)
    #display(leagueTable)
    posHome = df.apply(lambda row: pos(df, row.home, leagueTable), axis = 1)
    posAway = df.apply(lambda row: pos(df, row.away, leagueTable), axis = 1)
    df['avg_pos_home'] = posHome
    df['avg_pos_away'] = posAway
    return df

def table(df):
    teams = df['home'].unique()
    table = pd.DataFrame(teams, columns = ['teams'])
    pointTally = []
    for team in teams:
        pointTally.append(points(team, df))
    table['points'] = pointTally
    table = table.sort_values(by = 'points', ascending=False).reset_index(drop = True)
    return table

def points(team, df):
    homeGames = df.query('home == @team')['results'].replace(['H', 'D', 'A'], [3, 1, 0])
    awayGames = df.query('away == @team')['results'].replace(['H', 'D', 'A'], [0, 1, 3])
    totalGames = pd.concat([homeGames, awayGames], axis = 0)  
    avgPoints = totalGames.mean()
    return avgPoints

In [33]:
keep = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 
        'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR' ]
newCols = ['date', 'home', 'away', 'home_goals', 'away_goals', 'results', 'ht_home_goals', 'ht_away_goals', 
           'ht_result', 'home_shot', 'away_shot', 'home_sot', 'away_sot', 'home_foul', 'away_foul', 
           'home_corner', 'away_corner', 'home_yellow', 'away_yellow', 'home_red', 'away_red']

In [34]:
def localData(league, file):
    finaldf = pd.DataFrame()
    for i in range(5):
        if i == 0:
            df = pd.read_csv('Datasets/' + file + '.csv')
        else:    
            df = pd.read_csv('Datasets/' + file + ' (' + str(i) +').csv')

        df = df[keep]
        df.columns = newCols
        
        display(df)

        try:
            df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y')
        except ValueError:
            df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%y')

        df = addForm(df)

        df = avgPos(df)
        
        df = runningAverage(df)
        
        df = average(df)
        
        df = ratio(df)
        
        df = df.fillna(0)
        
        print('0')
        
        df['avg_pos'] = df['avg_pos_home'] / df['avg_pos_away']
        
        print('1')
    
        df['avg_pos_diff'] = df['avg_pos_home'] - df['avg_pos_away']
        print('2')
    
        df['5_form'] = df['home_5_form'] - df['away_5_form']
        print('3')
    
        df['3_form'] = df['home_3_form'] - df['away_3_form']
        print('4')
    
        df['1_form'] = df['home_1_form'] - df['away_1_form']
        print('5')
        
        df['results'] = df['results'].replace(['H', 'D', 'A'], range(3))
        print('6')
    
        df['ht_result'] = df['ht_result'].replace(['H', 'D', 'A'], range(3))
        print('7')
        
        display(df)
    
        
        finaldf = pd.concat([finaldf, df])
    
    finaldf.to_csv('local_' + league + '_processed.csv')
        
    return df

In [35]:
leagues = {
    'prem': ['E0'],
    'ligue': ['F1'],
    'bundes': ['D1'], 
    'serie': ['I1'],
    'laliga': ['SP1']}

for league in leagues:
    file = leagues[league][0]
    df = localData(league, file)

,date,home,away,home_goals,away_goals,results,ht_home_goals,ht_away_goals,ht_result,home_shot,...,home_sot,away_sot,home_foul,away_foul,home_corner,away_corner,home_yellow,away_yellow,home_red,away_red
0,13/08/2021,Brentford,Arsenal,2,0,H,1,0,H,8,...,3,4,12,8,2,5,0,0,0,0
1,14/08/2021,Man United,Leeds,5,1,H,1,0,H,16,...,8,3,11,9,5,4,1,2,0,0
2,14/08/2021,Burnley,Brighton,1,2,A,1,0,H,14,...,3,8,10,7,7,6,2,1,0,0
3,14/08/2021,Chelsea,Crystal Palace,3,0,H,2,0,H,13,...,6,1,15,11,5,2,0,0,0,0
4,14/08/2021,Everton,Southampton,3,1,H,0,1,A,14,...,6,3,13,15,6,8,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,07/05/2022,Liverpool,Tottenham,1,1,D,0,0,D,22,...,3,3,12,8,11,5,3,2,0,0
348,08/05/2022,Arsenal,Leeds,2,1,H,2,0,H,19,...,9,2,13,15,8,2,2,2,0,1
349,08/05/2022,Leicester,Everton,1,2,A,1,2,A,16,...,9,4,11,6,6,1,3,0,0,0
350,08/05/2022,Norwich,West Ham,0,4,A,0,3,A,8,...,2,5,7,7,9,5,1,1,0,0


0
1
2
3
4
5
6
7


,date,home,away,home_goals,away_goals,results,ht_home_goals,ht_away_goals,ht_result,home_shot,...,away_shot_attack,home_sot_attack,away_sot_attack,home_corner_attack,away_corner_attack,avg_pos,avg_pos_diff,5_form,3_form,1_form
0,2021-08-13,Brentford,Arsenal,2,0,0,1,0,0,8,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.500000,10,0,0,0
1,2021-08-14,Man United,Leeds,5,1,0,1,0,0,16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.352941,-11,0,0,0
2,2021-08-14,Burnley,Brighton,1,2,2,1,0,0,14,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,9,0,0,0
3,2021-08-14,Chelsea,Crystal Palace,3,0,0,2,0,0,13,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.272727,-8,0,0,0
4,2021-08-14,Everton,Southampton,3,1,0,0,1,2,14,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.066667,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,2022-05-07,Liverpool,Tottenham,1,1,1,0,0,1,22,...,83.294118,29.411765,11.470588,42.823529,14.754325,0.400000,-3,5,2,2
348,2022-05-08,Arsenal,Leeds,2,1,0,2,0,0,19,...,119.294118,36.961938,13.266436,30.730104,18.179931,0.235294,-13,2,-4,2
349,2022-05-08,Leicester,Everton,1,2,2,1,2,2,16,...,173.406250,24.960938,15.539062,29.773438,19.500000,0.750000,-4,11,7,1
350,2022-05-08,Norwich,West Ham,0,4,2,0,3,2,8,...,147.179931,16.041522,21.695502,27.404844,28.678201,2.857143,13,2,3,0


,date,home,away,home_goals,away_goals,results,ht_home_goals,ht_away_goals,ht_result,home_shot,...,home_sot,away_sot,home_foul,away_foul,home_corner,away_corner,home_yellow,away_yellow,home_red,away_red
0,12/09/2020,Fulham,Arsenal,0,3,A,0,1,A,5,...,2,6,12,12,2,3,2,2,0,0
1,12/09/2020,Crystal Palace,Southampton,1,0,H,1,0,H,5,...,3,5,14,11,7,3,2,1,0,0
2,12/09/2020,Liverpool,Leeds,4,3,H,3,2,H,22,...,6,3,9,6,9,0,1,0,0,0
3,12/09/2020,West Ham,Newcastle,0,2,A,0,0,D,15,...,3,2,13,7,8,7,2,2,0,0
4,13/09/2020,West Brom,Leicester,0,3,A,0,0,D,7,...,1,7,12,9,2,5,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,23/05/2021,Liverpool,Crystal Palace,2,0,H,1,0,H,19,...,5,4,10,8,14,1,2,2,0,0
376,23/05/2021,Man City,Everton,5,0,H,2,0,H,21,...,11,3,8,10,7,5,2,2,0,0
377,23/05/2021,Sheffield United,Burnley,1,0,H,1,0,H,12,...,3,3,11,1,8,9,3,1,0,0
378,23/05/2021,West Ham,Southampton,3,0,H,2,0,H,14,...,7,5,5,9,2,3,0,3,0,0


0
1
2
3
4
5
6
7


...

0
1
2
3
4
5
6
7


,date,home,away,home_goals,away_goals,results,ht_home_goals,ht_away_goals,ht_result,home_shot,...,away_shot_attack,home_sot_attack,away_sot_attack,home_corner_attack,away_corner_attack,avg_pos,avg_pos_diff,5_form,3_form,1_form
0,2018-08-17,Betis,Levante,0,3,2,0,1,2,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,-5,0,0,0
1,2018-08-17,Girona,Valladolid,0,0,1,0,0,1,13,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.125000,2,0,0,0
2,2018-08-18,Barcelona,Alaves,3,0,0,0,0,1,25,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,-8,0,0,0
3,2018-08-18,Celta,Espanol,1,1,1,0,1,2,12,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.428571,10,0,0,0
4,2018-08-18,Villarreal,Sociedad,1,2,2,1,1,1,16,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.272727,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2019-05-18,Levante,Ath Madrid,2,2,1,2,0,0,17,...,159.567901,21.111111,16.907407,30.694444,25.277778,7.500000,13,3,4,3
376,2019-05-18,Sevilla,Ath Bilbao,2,0,0,1,0,0,9,...,85.802469,18.438272,12.500000,24.074074,12.098765,0.750000,-2,3,3,0
377,2019-05-18,Valladolid,Valencia,0,2,2,0,1,2,19,...,111.416667,19.098765,13.222222,32.222222,17.783951,4.000000,12,-1,1,0
378,2019-05-19,Eibar,Barcelona,2,2,1,2,2,1,15,...,98.904321,21.111111,18.574074,37.123457,14.283951,12.000000,11,1,2,3


,date,home,away,home_goals,away_goals,results,ht_home_goals,ht_away_goals,ht_result,home_shot,...,home_sot,away_sot,home_foul,away_foul,home_corner,away_corner,home_yellow,away_yellow,home_red,away_red
0,18/08/17,Leganes,Alaves,1,0,H,1,0,H,16,...,9,3,14,18,4,2,0,1,0,0
1,18/08/17,Valencia,Las Palmas,1,0,H,1,0,H,22,...,6,4,25,13,5,2,3,3,0,1
2,19/08/17,Celta,Sociedad,2,3,A,1,1,D,16,...,5,6,12,11,5,4,3,1,0,0
3,19/08/17,Girona,Ath Madrid,2,2,D,2,0,H,13,...,6,3,15,15,6,0,2,4,0,1
4,19/08/17,Sevilla,Espanol,1,1,D,1,1,D,9,...,4,6,14,12,7,3,2,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,19/05/18,Villarreal,Real Madrid,2,2,D,0,2,A,14,...,5,7,13,10,4,5,4,3,0,0
376,20/05/18,Ath Bilbao,Espanol,0,1,A,0,1,A,12,...,3,4,4,11,7,7,0,1,0,0
377,20/05/18,Ath Madrid,Eibar,2,2,D,1,1,D,8,...,4,3,14,14,2,2,6,2,1,0
378,20/05/18,Barcelona,Sociedad,1,0,H,0,0,D,12,...,4,2,13,13,3,8,3,2,0,0


0
1
2
3
4
5
6
7


,date,home,away,home_goals,away_goals,results,ht_home_goals,ht_away_goals,ht_result,home_shot,...,away_shot_attack,home_sot_attack,away_sot_attack,home_corner_attack,away_corner_attack,avg_pos,avg_pos_diff,5_form,3_form,1_form
0,2017-08-18,Leganes,Alaves,1,0,0,1,0,0,16,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.214286,3,0,0,0
1,2017-08-18,Valencia,Las Palmas,1,0,0,1,0,0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.210526,-15,0,0,0
2,2017-08-19,Celta,Sociedad,2,3,2,1,1,1,16,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.181818,2,0,0,0
3,2017-08-19,Girona,Ath Madrid,2,2,1,2,0,0,13,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,8,0,0,0
4,2017-08-19,Sevilla,Espanol,1,1,1,1,1,1,9,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.583333,-5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2018-05-19,Villarreal,Real Madrid,2,2,1,0,2,2,14,...,177.160494,21.854938,23.716049,25.601852,28.858025,1.666667,2,2,5,3
376,2018-05-20,Ath Bilbao,Espanol,0,1,2,0,1,2,12,...,93.521605,20.953704,11.287037,31.135802,15.814815,1.333333,4,1,-3,0
377,2018-05-20,Ath Madrid,Eibar,2,2,1,1,1,1,8,...,112.907407,18.250000,9.225309,28.092593,20.283951,0.222222,-7,3,-3,-3
378,2018-05-20,Barcelona,Sociedad,1,0,0,0,0,1,12,...,136.410494,35.679012,12.617284,30.419753,19.638889,0.090909,-10,9,4,3
